In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import os
from model.expression_embedding import expression_GVAE
from common.data_processing import data_preprocessing_ST1K
from common.dataset import CLIPDataset
import torch
import train

In [ ]:
input_path='/home2/mengfanyi/YW/HE_predict/data/HEST/COLON-CANCER/'
raw_adata_list1, raw_HE_image_paths1,raw_nuclei_mask_paths1=data_preprocessing_ST1K(input_path)
raw_names1 = [d[:-5] for d in os.listdir(input_path) if d.endswith('.h5ad')]

input_path='/home2/mengfanyi/YW/HE_predict/data/HEST/COLON-HEALTHY/'
raw_adata_list2, raw_HE_image_paths2,raw_nuclei_mask_paths2=data_preprocessing_ST1K(input_path)
raw_names2 = [d[:-5] for d in os.listdir(input_path) if d.endswith('.h5ad')]

adata_list=raw_adata_list1+raw_adata_list2
HE_image_paths=raw_HE_image_paths1+raw_HE_image_paths2
nuclei_mask_paths=raw_nuclei_mask_paths1+raw_nuclei_mask_paths2
names=raw_names1+raw_names2

save_dir='./data/HEST/COLON-CANCER-HEALTHY/'

print(names)
print(len(names))
print(len(adata_list))
print(len(HE_image_paths))

In [ ]:
for i in range(len(adata_list)):
    sc.pp.filter_genes(adata_list[i], min_cells=1)
    sc.pp.filter_cells(adata_list[i], min_genes=1)
    adata_list[i].var_names_make_unique()

shared_var_names = set(adata_list[0].var_names)
for adata in adata_list[1:]:
    shared_var_names.intersection_update(adata.var_names)
print(len(shared_var_names))


for i in range(len(adata_list)):
    adata_list[i] = adata_list[i][:, list(shared_var_names)]
    
print(adata_list[0])

In [ ]:
Input_Method='hvg'
new_adata_list=expression_GVAE(adatas=adata_list,method='GATE',feature_method=Input_Method,n_top_genes=1000)
print(new_adata_list[0])

In [ ]:
print(new_adata_list[0].obsm['embedding_rep'].shape)
image_embedding=new_adata_list[0].obsm['embedding_rep'].shape[1]
out_embedding=new_adata_list[0].X.shape[1]
print(out_embedding)

In [ ]:
dataset_list=[]
for i in range(len(new_adata_list)):
    if 'block_r' in adata_list[i].uns:
        new_adata_list[i].uns['block_r']=adata_list[i].uns['block_r']
        print(new_adata_list[i].uns['block_r'])
    dataset = CLIPDataset(image_path = HE_image_paths[i],ST_adata = new_adata_list[i],nuclei_mask_path=nuclei_mask_paths[i])
    dataset_list.append(dataset)
print(dataset_list)

In [ ]:
# Leave-one-out cross validation
train_index=[i for i in range(len(new_adata_list))]
val_index=None

for i in train_index: 
    test_index=[i]
    new_train_index = [item for item in train_index if item not in test_index]
    train_dataset = torch.utils.data.ConcatDataset([dataset_list[i] for i in new_train_index])
    val_dataset = None
    test_dataset = torch.utils.data.ConcatDataset([dataset_list[i] for i in test_index])
    print(names[test_index[0]])
    savepath=os.path.join(save_dir,names[test_index[0]],'results/')
    if not os.path.exists(savepath):
        os.makedirs(savepath)

    best_predict=train.main(train_dataset, val_dataset, test_dataset, image_embedding, 
                        out_embedding, max_epochs=30, save_path=savepath)
    predict_adata=new_adata_list[test_index[0]].copy()
    predict_adata.X=best_predict[0].copy()

    gt_adata=new_adata_list[test_index[0]].copy()
    predict_adata.write_h5ad(os.path.join(savepath,'my_pre.h5ad'))
    gt_adata.write_h5ad(os.path.join(savepath,'my_gt.h5ad'))